<a href="https://colab.research.google.com/github/unt-iialab/INFO5731_Spring2020/blob/master/Assignments/INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [1]:
# Write your code here

# Getting data from Assignment 02.
import pandas as pd
data = pd.read_csv(r"C:\Users\Raheyma Arshad\Desktop\Output_CSV.csv")
data = data[['review', 'cleaned_review']]

# Tokenization of cleaned reviews.
import nltk
data['cleaned_review'] = data['cleaned_review'].apply(lambda x: nltk.word_tokenize(x))

# (1) Count the frequency of all the N-grams (N=3).

from nltk.util import ngrams
data['3-grams'] = data['cleaned_review'].apply(lambda row: list(nltk.ngrams(row, 3)))
data['Count of 3-grams'] = data['3-grams'].str.len()

# (2) Calculate the probabilities for all the bigrams in the dataset.

def probability(x):
    probabilities = []
    bigrams = list(nltk.ngrams(x, 2))
    for (w1, w2) in bigrams:
        numerator = bigrams.count((w1, w2))
        for w1 in (w1, w2):
            denominator = x.count(w1)
            probabilities.append(((w1, w2), numerator/denominator))
    return probabilities

data['Probability of bigrams'] = data['cleaned_review'].apply(lambda x: probability(x))
data.head(5)

,review,cleaned_review,3-grams,Count of 3-grams,Probability of bigrams
0,"Every once in a while a movie comes, that trul...","[everi, come, truli, make, impact, joaquin, pe...","[(everi, come, truli), (come, truli, make), (t...",52,"[((everi, come), 1.0), ((come, come), 1.0), ((..."
1,This is a movie that only those who have felt ...,"[thi, felt, alon, isol, truli, relat, you, und...","[(thi, felt, alon), (felt, alon, isol), (alon,...",46,"[((thi, felt), 1.0), ((felt, felt), 1.0), ((fe..."
2,"Truly a masterpiece, The Best Hollywood film o...","[truli, masterpiec, the, best, hollywood, film...","[(truli, masterpiec, the), (masterpiec, the, b...",91,"[((truli, masterpiec), 0.5), ((masterpiec, mas..."
3,Joaquin Phoenix gives a tour de force performa...,"[joaquin, phoenix, give, tour, de, forc, perfo...","[(joaquin, phoenix, give), (phoenix, give, tou...",58,"[((joaquin, phoenix), 1.0), ((phoenix, phoenix..."
4,Most of the time movies are anticipated like t...,"[most, time, movi, anticip, like, end, fall, s...","[(most, time, movi), (time, movi, anticip), (m...",58,"[((most, time), 1.0), ((time, time), 0.5), ((t..."


In [2]:
# (3) Extract all the noun phrases and calculate the relative probabilities of each review in terms of other reviews.

import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()

def noun_phrases(x):
    x = x.replace('[^\w\s]','')
    x = x.lower()
    nps = []
    doc = nlp(x)
    chunks = list(doc.noun_chunks)
    for chunk in chunks:
        nps.append(chunk)
    return nps

df = pd.DataFrame()
df['Reviews'] = data['review']
df['Noun Phrases'] = df['Reviews'].apply(lambda x: noun_phrases(x))
df['Noun Phrases'] = df['Noun Phrases'].apply(lambda x: [str(item) for item in x])
np_list = df["Noun Phrases"].tolist()
np_flat_list = [str(item) for sublist in np_list for item in sublist]

from collections import Counter
np_freq = Counter(np_flat_list)
print('max frequency (noun phrase) is:', np_freq.most_common(1)[0])
max_freq = np_freq.most_common(1)[0][1]

def relative_prob(x):
    phrase_prob = []
    phrases = []
    for phrase in x:
        if phrase not in phrases:
            freq = x.count(phrase)
            rel_prob = round(freq/max_freq, 5)
            phrase_prob.append((phrase, rel_prob))
            phrases.append(phrase)
    return phrase_prob

df['Relative Probabilities'] = df['Noun Phrases'].apply(lambda x: relative_prob(x))
df.head(5)

C:\Raheyma\Software\Anaconda\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


max frequency (noun phrase) is: ('it', 348)


,Reviews,Noun Phrases,Relative Probabilities
0,"Every once in a while a movie comes, that trul...","[a while, a movie, an impact, joaquin's perfor...","[(a while, 0.00287), (a movie, 0.00287), (an i..."
1,This is a movie that only those who have felt ...,"[a movie, who, it, you, the motive, you, the c...","[(a movie, 0.00287), (who, 0.00287), (it, 0.00..."
2,"Truly a masterpiece, The Best Hollywood film o...","[truly a masterpiece, the best hollywood film,...","[(truly a masterpiece, 0.00287), (the best hol..."
3,Joaquin Phoenix gives a tour de force performa...,"[joaquin phoenix, a tour de force performance,...","[(joaquin phoenix, 0.00287), (a tour de force ..."
4,Most of the time movies are anticipated like t...,"[the time, movies, they, joker, the first time...","[(the time, 0.00287), (movies, 0.00287), (they..."


# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [3]:
# (1) Building the documents-terms weights (tf*idf) matrix

# Creating a corpus of reviews.
import pandas as pd
reviews = pd.read_csv(r"C:\Users\Raheyma Arshad\Desktop\Output_CSV - Copy.csv")
corpus = reviews.set_index('review_id').T.to_dict('list')

def listToString(s):      
    # initialize an empty string 
    str1 = " " 
    # return string   
    return (str1.join(s)) 

corpus = {doc_label: listToString(s) for doc_label, s in corpus.items()}

import nltk

# Tokenize whole corpus
tokens = {doc_label: nltk.word_tokenize(text) for doc_label, text in corpus.items()}

# Count the tokens for each document
from collections import Counter
counts = {doc_label: Counter(tok) for doc_label, tok in tokens.items()}

# Extract the vocabulary (set of unique terms in all documents)
vocab = set()
for counter in counts.values():
    vocab |= set(counter.keys())    
vocab = sorted(list(vocab))

# Create Bag of Words matrix: rows are documents, columns are vocabulary words.
bow = []
for counter in counts.values():
    bow_row = [counter.get(term, 0) for term in vocab]
    bow.append(bow_row)
doc_labels = list(counts.keys())

# Term Frequency
import numpy as np
raw_counts = np.mat(bow, dtype=float)       
tf = raw_counts / np.sum(raw_counts, axis=1)

# idf – inverse document frequency
def num_term_in_docs(t, docs):
    return sum(t in d for d in docs.values())

from math import log
def idf(t, docs):
    a = num_term_in_docs(t, docs)
    return log(1 + len(docs) / (1+a))

idf_row = [idf(t, tokens) for t in vocab]
idf_mat = np.mat(np.diag(idf_row))
tfidf = tf * idf_mat

# Presenting the tf-idf matrix as a dataframe
tfidf_arrays = np.array(tfidf)
tfidf_df = pd.DataFrame(data=tfidf_arrays, index=reviews['review_id'], columns=vocab)
tfidf_df.head()

,a,aaah,abl,absolut,absurd,abus,accept,acclaim,accolad,accompli,...,you,youd,youll,young,younger,your,youv,zazi,zero,zimmer
review_id,,,,,,,,,,,,,,,,,,,,,
rev1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rev2,0.039173,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.051961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rev3,0.020218,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rev4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rev5,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# (2) To rank the documents with respect to query by using cosine similarity.

from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.stem import PorterStemmer
from textblob import Word

def clean_query(query):
    query = query.replace('[^\w\s]','')
    query = query.replace('\d+', '')
    query = " ".join(x for x in query.split() if x not in stop)
    query = " ".join(x.lower() for x in query.split())
    query = " ".join([PorterStemmer().stem(word) for word in query.split()])
    query = " ".join([Word(word).lemmatize() for word in query.split()])
    return query

def tfidf_vector_of_query(query):
    q_corpus = {'query': clean_query(query)}
    
    q_tokens = {doc_label: nltk.word_tokenize(text) for doc_label, text in q_corpus.items()}
    q_counts = {doc_label: Counter(tok) for doc_label, tok in q_tokens.items()}

    q_bow = []
    for counter in q_counts.values():
        q_bow_row = [counter.get(term, 0) for term in vocab]
        q_bow.append(q_bow_row)
    q_label = list(q_counts.keys())

    q_raw_counts = np.mat(q_bow, dtype=float)       
    q_tf = q_raw_counts / np.sum(q_raw_counts, axis=1)

    q_idf_row = [idf(t, q_tokens) for t in vocab]
    q_idf_mat = np.mat(np.diag(q_idf_row))
    q_tfidf = q_tf * q_idf_mat

    q_tfidf_array = np.array(q_tfidf)
    return q_tfidf_array

from numpy import linalg as la

def cosine_similarity(query, rev):
    rev = np.array(rev)
    query = tfidf_vector_of_query(query)
    if la.norm(query) != 0 and la.norm(rev) != 0:
        cos_sim = np.inner(query, rev) / la.norm(query) * la.norm(rev)
    else:
        cos_sim = 0
    return float(cos_sim)

query = "An Outstanding movie with a haunting performance and best character development"

cos_sim_list = []
for i in range(len(tfidf_df)):
    cos_sim = cosine_similarity(query, tfidf_df.iloc[i])
    cos_sim_list.append(cos_sim)

cos_sim_df = pd.DataFrame({'review_id': doc_labels, 'Cosine Similarity with Query': cos_sim_list})
cos_sim_df.head()

,review_id,Cosine Similarity with Query
0,rev1,0.028861
1,rev2,0.004285
2,rev3,0.005668
3,rev4,0.006743
4,rev5,0.003087


# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [0]:
# The GitHub link of your final csv file

# Link: https://github.com/raheymakhan/Assignment-03/blob/main/Annotated%20Data.csv